**Component buildup method to evaluate the parasitic drag coefficient $C_{D0}$**

In [2]:
import numpy as np

- Input data

In [3]:
# FLIGHT CONDITIONS
V = 18.0 # m/s

# FUSELAGE
fus_width = 0.25
fus_height = 0.25
fus_length = 2.2

# WING
w_t_c = 0.125 # wing thickness-to-chord ratio
w_x_c_max = 0.237 # wing max thickness location
w_chord = 0.50
w_S = 2.0

# HORIZONTAL TAILPLANE
ht_t_c = 0.12
ht_x_c_max = 0.3
ht_chord = 0.41
ht_S = 0.66

# VERTICAL TAILPLANE
vt_t_c = 0.12
vt_x_c_max = 0.3
vt_chord = 0.43
vt_S = 0.37

# SKIN ROUGHNESS
skin_roughness = 0.634e-5 # [metre], for smooth paint (Raymer)

- Useful functions

In [ ]:
# laminar friction coefficient
def C_f_lam(Re):
    return 1.328/np.sqrt(Re)

# turbulent friction coefficient
def C_f_turb(Re):
    return 0.0442/(Re**1/6)

# Reynolds number at sea level conditions
def Re(V, l):
    return 1.225*V*l/1.8e-5

# cutoff Reynolds number
def Re_cutoff (l, k):
    return 38.21 * (l/k)**1.053

# lifting surface, pylon, strut, form factor
def FF_wing(x_c_max, t_c):
    return 1 + 0.6/x_c_max * t_c + 100 * t_c**4

# fuselage form factor
def FF_fus(length, front_area, geometry: str='circ', ):
    d_eq = np.sqrt(4*front_area/np.pi) # equivalent diameter
    f = length/d_eq
    FF = 0.9 + 5/f**1.5 + f/400
    if geometry == 'square':
        return FF*1.35 # Raymer : 30-40% extra FF for square-sided fuselage
    elif geometry == 'circ':
        return FF
    else :
        return FF

# wing wetted area
def S_wet_wing(S_exposed, t_c):
    return S_exposed * (1.977 + 0.52*t_c)


- Friction drag

In [1]:
# WING
w_Re_cutoff = Re_cutoff(w_chord, skin_roughness)
w_Re = Re(V, w_chord)